In [ ]:
%cd ..
%cd ..

In [ ]:
!pip install -r requirements.txt
# !pip install -U pydantic

In [ ]:
# !pip uninstall pydantic -y
# !pip install pydantic==2.10.6
!pip show pydantic

In [ ]:
import sys
sys.path.append(r"C:\Users\amate\GIT\game_demo")

In [ ]:
import os
print(os.getcwd())

In [ ]:
"""
app/services/game.py
GameService - 낮 파이프라인 통합

DB(Games 모델) → WorldStatePipeline 변환 → 파이프라인 실행 → DB 저장
파이프라인: LockManager → DayController → EndingChecker → NarrativeLayer
"""
# from __future__ import annotations
import logging

logging.basicConfig(level=logging.DEBUG, force=True)

import copy
from typing import Any, Dict
# from app.crud import game as crud_game
# from app.redis_client import get_redis_client

from app.schemas.client_sync import GameClientSyncSchema
from app.schemas.world_meta_data import WorldDataSchema, LocksSchemaList
from app.schemas.npc_info import NpcCollectionSchema
from app.schemas.player_info import PlayerSchema
from app.schemas.player_info import PlayerSchema
from app.day_controller import get_day_controller
from app.loader import ScenarioLoader, ScenarioAssets
from pathlib import Path

from sqlalchemy.orm import Session
from sqlalchemy.orm.attributes import flag_modified

from app.db_models.game import Games
from app.db_models.scenario import Scenario
# from app.crud import game as crud_game
from app.loader import ScenarioAssets
from app.schemas.game_state import NPCState, WorldStatePipeline, StateDelta
from app.schemas.tool import ToolResult
from app.lock_manager import get_lock_manager
from app.ending_checker import check_ending
from app.narrative import get_narrative_layer
from app.day_controller import get_day_controller
from app.night_controller import get_night_controller
import logging

logger=logging.getLogger(__name__)

# ============================================================
# Delta 적용 로직
# ============================================================

from app.services.game import (
    _apply_delta,
    mock_load_scenario_assets_from_yaml,
    mock_create_world_state_from_yaml,
)


In [ ]:
"""
process_turn() 하나씩 뜯어보기
목업 Assets / WorldStatePipeline 생성
"""

# STEP 1 : Assets 불러오기 (목업)
assets = mock_load_scenario_assets_from_yaml()

def print_pydantic(data):
    """Pydantic BaseModel, dataclass, dict 등 어떤 타입이든 JSON으로 출력"""
    import json
    from dataclasses import asdict, is_dataclass
    from enum import Enum

    # 1) Pydantic BaseModel → mode="json" (set→list, Enum→value 자동 변환)
    if hasattr(data, "model_dump"):
        dumped = data.model_dump(mode="json")
    # 2) dataclass
    elif is_dataclass(data) and not isinstance(data, type):
        dumped = asdict(data)
    # 3) dict / list 등
    elif isinstance(data, (dict, list)):
        dumped = data
    else:
        dumped = str(data)
        print(dumped)
        return

    # fallback encoder: Enum, set 등 json.dumps가 못 처리하는 타입 대비
    def _default(obj):
        if isinstance(obj, set):
            return sorted(obj)
        if isinstance(obj, Enum):
            return obj.value
        if hasattr(obj, "model_dump"):
            return obj.model_dump(mode="json")
        if is_dataclass(obj) and not isinstance(obj, type):
            return asdict(obj)
        return str(obj)

    print(json.dumps(dumped, ensure_ascii=False, indent=2, default=_default))

print_pydantic(assets)

In [ ]:
assets.extras

In [ ]:
# STEP 2 : WorldStatePipeline 불러오기 (목업)
# 초기화 코드

world_state = mock_create_world_state_from_yaml()
print_pydantic(world_state)

In [ ]:
world_state.flags

In [ ]:
# ── Step 3: LockManager - 정보 해금 ──

# 강제 해금 - topic_brother_injury (npc.brother.affection >= 70)
world_state.npcs["brother"].stats["affection"] = 80
print(f"변경된 brother의 affection 수치 : {world_state.npcs['brother'].stats['affection']}")

# 강제 해금 - lore_decay_scent (npc.dog_baron.affection >= 80)
world_state.npcs["dog_baron"].stats["affection"] = 80
print(f"변경된 dog_baron affection 수치 : {world_state.npcs['brother'].stats['affection']}")

lock_manager = get_lock_manager()
locks_data = assets.extras.get("locks", {})
lock_result = lock_manager.check_unlocks(world_state, locks_data)
print_pydantic(lock_result)

In [ ]:
# NPC 메모리 갱신 확인
world_state.npcs["brother"].memory, world_state.npcs["dog_baron"].memory

In [ ]:
# 해금 여부 저장되었는지 확인
world_state.locks

# 낮 파이프라인 테스트

In [ ]:
print_pydantic(world_state)

In [ ]:
# ── Step 3.5: StatusEffectManager - 만료 효과 해제 ──
# 임의로 설정
from app.schemas.item_use import StatusEffect
from app.schemas.status import NPCStatus

world_state.vars["status_effects"].append(StatusEffect(
    target_npc_id="stepmother",
    applied_status=NPCStatus.SLEEPING,
    expires_at_turn=2,
))
print_pydantic(world_state.vars["status_effects"])

In [ ]:
# 해제 확인
world_state.turn = 2

from app.status_effect_manager import get_status_effect_manager
sem = get_status_effect_manager()
sem.tick(current_turn=world_state.turn, world_state=world_state)

# 확인: vars status_effects 키가 생겼는지
print_pydantic(world_state.vars["status_effects"])
print(f"stepmother status: {world_state.npcs['stepmother'].status}")

In [ ]:
world_state.npcs["stepmother"].current_phase_id = 'B'
print(world_state.npcs["stepmother"])

In [ ]:
user_text = " 새엄마에게 집에서 나가고 싶다고 말한다."
logging.basicConfig(level=logging.INFO, force=True)
# ── Step 4: DayController - 낮 턴 실행 ──
day_controller = get_day_controller()
tool_result: ToolResult = day_controller.process(
    user_text,
    world_state,
    assets,
)
tool_result

In [ ]:
# 새엄마의 plus, minus에 해당하는대로 말해보기

user_text = "루카스 같이 도망치자"

day_controller = get_day_controller()
tool_result: ToolResult = day_controller.process(
    user_text,
    world_state,
    assets,
)
tool_result

In [ ]:
# ── Step 5: Delta 적용 ──
world_after = _apply_delta(world_state, tool_result.state_delta, assets)
print_pydantic(world_after)

In [ ]:
world_state.locks['quest_fire_weakness'] = True

In [ ]:
print_pydantic(world_state)

In [ ]:
# 유저 위치 수동 설정 (말린 허브는 지하실에 있으므로)
world_state.player_location = "basement"
print(f"player_location: {world_state.player_location}")

In [ ]:
# 아이템 얻어보기 (수동)

user_text = "새엄마에게 라이터를 사용한다"

day_controller = get_day_controller()
tool_result: ToolResult = day_controller.process(
    user_text,
    world_state,
    assets,
)
tool_result

world_after = _apply_delta(world_state, tool_result.state_delta, assets)
print_pydantic(world_after)

In [ ]:
# 아이템 얻어보기 (수동)

user_text = "불을 지른다"

day_controller = get_day_controller()
tool_result: ToolResult = day_controller.process(
    user_text,
    world_state,
    assets,
)
tool_result

world_after = _apply_delta(world_state, tool_result.state_delta, assets)
print_pydantic(world_after)

In [ ]:
# 아이템 얻어보기 (수동) - 막힌 아이템

user_text = "수면제를 줍는다."

day_controller = get_day_controller()
tool_result: ToolResult = day_controller.process(
    user_text,
    world_state,
    assets,
)
tool_result

world_after = _apply_delta(world_state, tool_result.state_delta, assets)
print_pydantic(world_after.inventory)

In [ ]:
# 아이템을 사용해보기 

user_text = "가족사진을 새엄마에게 보여준다."

day_controller = get_day_controller()
tool_result: ToolResult = day_controller.process(
    user_text,
    world_state,
    assets,
)
tool_result

In [ ]:
# ── Step 5.5: ItemAcquirer - 자동 아이템 획득 스캔 ──
# real_family_photo - npc.dog_baron.affection >= 90

# world_state.inventory.remove("real_family_photo")
world_state.npcs["dog_baron"].stats["affection"] = 95

from app.item_acquirer import get_item_acquirer
acquirer = get_item_acquirer()
acq_result = acquirer.scan(world_after, assets)
if acq_result.newly_acquired:
    world_after = _apply_delta(world_after, acq_result.acquisition_delta, assets)
    for acq_item_id in acq_result.newly_acquired:
        acq_item_def = assets.get_item_by_id(acq_item_id)
        acq_item_name = acq_item_def.get("name", acq_item_id) if acq_item_def else acq_item_id
        tool_result.event_description.append(f"'{acq_item_name}'을(를) 발견했다!")
print_pydantic(acq_result)

In [ ]:
# ── Step 5.6: day_action_log 축적 (밤 가족회의 안건용) ──
day_log_entry = {
    "turn": world_after.turn,
    "input": user_text,
    "intent": tool_result.intent,
    "events": tool_result.event_description,
}
world_after.vars.setdefault("day_action_log", []).append(day_log_entry)
day_log_entry

In [ ]:
# ── Step 6: EndingChecker - 엔딩 체크 ──
ending_result = check_ending(world_after, assets)
ending_info = None
if ending_result.reached:
    ending_info = {
        "ending_id": ending_result.ending.ending_id,
        "name": ending_result.ending.name,
        "epilogue_prompt": ending_result.ending.epilogue_prompt,
    }
    if ending_result.triggered_delta:
        _apply_delta(world_after, ending_result.triggered_delta, assets)
ending_info, ending_result

In [ ]:
# ── Step 7: NarrativeLayer - 나레이션 생성 ──
narrative_layer = get_narrative_layer()
if ending_info:
    narrative = narrative_layer.render_ending(
        ending_info,
        world_after,
        assets,
    )
else:
    narrative = narrative_layer.render(
        world_after,
        assets,
        event_description=tool_result.event_description,
        state_delta=tool_result.state_delta,
        npc_response=tool_result.npc_response,
    )
print(narrative)

In [ ]:
# 강제 엔딩 체크
## "조력자의 희생 (The Sibling's Help)"
## condition: "npc.brother.affection >= 90 and npc.brother.humanity >= 70 and vars.day == 5"

world_state.npcs["brother"].stats["affection"] = 95
world_state.npcs["brother"].stats["humanity"] = 80
world_state.vars["day"] = 5

ending_result = check_ending(world_state, assets)
ending_info = None
if ending_result.reached:
    ending_info = {
        "ending_id": ending_result.ending.ending_id,
        "name": ending_result.ending.name,
        "epilogue_prompt": ending_result.ending.epilogue_prompt,
    }
    if ending_result.triggered_delta:
        _apply_delta(world_after, ending_result.triggered_delta, assets)

print(f"ending_info : {ending_info}")
print(f"ending_result : {ending_result}")

In [ ]:
narrative_layer = get_narrative_layer()
if ending_info:
    narrative = narrative_layer.render_ending(
        ending_info,
        world_state,
        assets,
    )
else:
    narrative = narrative_layer.render(
        world_state,
        assets,
        event_description=tool_result.event_description,
        state_delta=tool_result.state_delta,
        npc_response=tool_result.npc_response,
    )
print(narrative)

In [ ]:
# ── Step 8: WorldStatePipeline → DB 반영 ──


In [ ]:
# Game 저장

In [ ]:
# 리턴

# 밤 파이프라인 테스트

In [ ]:
# # ── Step 4: NightController - 밤 턴 실행 ──
# night_controller = get_night_controller()
# night_result = night_controller.process(world_state, assets)
# night_result